In [64]:
import pandas as pd
from nltk.tokenize import word_tokenize
en_stopwords = nltk.corpus.stopwords.words('english')

In [1]:
from datasets import load_dataset

dataset = load_dataset("billsum")

Generating train split:   0%|          | 0/18949 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/3269 [00:00<?, ? examples/s]

Generating ca_test split:   0%|          | 0/1237 [00:00<?, ? examples/s]

Dataset billsum downloaded and prepared to /Users/emilygong/.cache/huggingface/datasets/billsum/default/3.0.0/75cf1719d38d6553aa0e0714c393c74579b083ae6e164b2543684e3e92e0c4cc. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

In [ ]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

tokenizer = AutoTokenizer.from_pretrained("google/pegasus-xsum")

model = AutoModelForSeq2SeqLM.from_pretrained("google/pegasus-xsum")

In [8]:
dataset

DatasetDict({
    train: Dataset({
        features: ['text', 'summary', 'title'],
        num_rows: 18949
    })
    test: Dataset({
        features: ['text', 'summary', 'title'],
        num_rows: 3269
    })
    ca_test: Dataset({
        features: ['text', 'summary', 'title'],
        num_rows: 1237
    })
})

In [11]:
train = pd.DataFrame(dataset['train'] )
train = train.dropna()
test = pd.DataFrame(dataset['test'] )
test = test.dropna()
ca_test = pd.DataFrame(dataset['ca_test'] )
ca_test = ca_test.dropna()

In [12]:
train

,text,summary,title
0,SECTION 1. LIABILITY OF BUSINESS ENTITIES PROV...,Shields a business entity from civil liability...,A bill to limit the civil liability of busines...
1,SECTION 1. SHORT TITLE.\n\n This Act may be...,Human Rights Information Act - Requires certai...,Human Rights Information Act
2,SECTION 1. SHORT TITLE.\n\n This Act may be...,Jackie Robinson Commemorative Coin Act - Direc...,Jackie Robinson Commemorative Coin Act
3,SECTION 1. NONRECOGNITION OF GAIN WHERE ROLLOV...,Amends the Internal Revenue Code to provide (t...,To amend the Internal Revenue Code to provide ...
4,SECTION 1. SHORT TITLE.\n\n This Act may be...,Native American Energy Act - (Sec. 3) Amends t...,Native American Energy Act
...,...,...,...
18944,SECTION 1. SHORT TITLE.\n\n This Act may be...,District of Columbia Economic Recovery Act - A...,District of Columbia Economic Recovery Act
18945,SECTION 1. RECONSULTATION ON CENTRAL VALLEY PR...,Requires the Commissioner of the Bureau of Rec...,To direct the Commissioner of the Bureau of Re...
18946,SECTION 1. CHARITABLE DEDUCTION FOR CONTRIBUTI...,Amends the Internal Revenue Code to set forth ...,A bill to amend the Internal Revenue Code of 1...
18947,SECTION 1. SPECIAL RULE FOR CERTAIN FACILITIES...,"This bill amends the Internal Revenue Code, wi...",A bill to amend the Internal Revenue Code of 1...


In [14]:
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.corpus import stopwords

In [65]:
example = train.iat[0, 0]
print(example)

SECTION 1. LIABILITY OF BUSINESS ENTITIES PROVIDING USE OF FACILITIES 
              TO NONPROFIT ORGANIZATIONS.

    (a) Definitions.--In this section:
            (1) Business entity.--The term ``business entity'' means a 
        firm, corporation, association, partnership, consortium, joint 
        venture, or other form of enterprise.
            (2) Facility.--The term ``facility'' means any real 
        property, including any building, improvement, or appurtenance.
            (3) Gross negligence.--The term ``gross negligence'' means 
        voluntary and conscious conduct by a person with knowledge (at 
        the time of the conduct) that the conduct is likely to be 
        harmful to the health or well-being of another person.
            (4) Intentional misconduct.--The term ``intentional 
        misconduct'' means conduct by a person with knowledge (at the 
        time of the conduct) that the conduct is harmful to the health 
        or well-being of another perso

In [76]:
import regex as re

def preprocessing(string):
    # symbols
    string = re.sub(r'\([^)]*\)', '', string)
    string = re.sub('\n', '', string)
    string = re.sub(' +', ' ', string)
    string = re.sub(r'[^\w\s]', '', string)
    string = string.lower()
    # tokenize
    tokenized_string = word_tokenize(string) 
    # remove stopwords
    filtered_words = [word for word in tokenized_string if word not in en_stopwords]
    return filtered_words

In [77]:
print(preprocessing(example))

['section', '1', 'liability', 'business', 'entities', 'providing', 'use', 'facilities', 'nonprofit', 'organizations', 'definitionsin', 'section', 'business', 'entitythe', 'term', 'business', 'entity', 'means', 'firm', 'corporation', 'association', 'partnership', 'consortium', 'joint', 'venture', 'form', 'enterprise', 'facilitythe', 'term', 'facility', 'means', 'real', 'property', 'including', 'building', 'improvement', 'appurtenance', 'gross', 'negligencethe', 'term', 'gross', 'negligence', 'means', 'voluntary', 'conscious', 'conduct', 'person', 'knowledge', 'conduct', 'likely', 'harmful', 'health', 'wellbeing', 'another', 'person', 'intentional', 'misconductthe', 'term', 'intentional', 'misconduct', 'means', 'conduct', 'person', 'knowledge', 'conduct', 'harmful', 'health', 'wellbeing', 'another', 'person', 'nonprofit', 'organizationthe', 'term', 'nonprofit', 'organization', 'means', 'organization', 'described', 'section', '501', 'internal', 'revenue', 'code', '1986', 'exempt', 'tax', 